In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10_diff_norm_value_n_grayscale"
SETTINGS = [
    {
        "normalization_mean": None,
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.774, 0.439, 0.859],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.697, 0.094, 0.976],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.761, 0.786, 0.128],
        "normalization_std": None,
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.3471, 0.3435, 0.3616],
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.2971, 0.2935, 0.3116],
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.2721, 0.2685, 0.2866],
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.1471, 0.1435, 0.1616],
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.1971, 0.1935, 0.2116],
    },
    {
        "normalization_mean": None,
        "normalization_std": [0.2221, 0.2185, 0.2366],
    },
]


In [3]:
extra_preprocessing_type = "rgb-grayscale"

for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_std_{std_str}"
    else:
        label = f"model_{extra_preprocessing_type}_baseline"

    print(label)

model_rgb-grayscale_baseline
model_rgb-grayscale_mean_0.774_0.439_0.859
model_rgb-grayscale_mean_0.697_0.094_0.976
model_rgb-grayscale_mean_0.761_0.786_0.128
model_rgb-grayscale_std_0.3471_0.3435_0.3616
model_rgb-grayscale_std_0.2971_0.2935_0.3116
model_rgb-grayscale_std_0.2721_0.2685_0.2866
model_rgb-grayscale_std_0.1471_0.1435_0.1616
model_rgb-grayscale_std_0.1971_0.1935_0.2116
model_rgb-grayscale_std_0.2221_0.2185_0.2366


In [4]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [9]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_{extra_preprocessing_type}_std_{std_str}"
    else:
        label = f"model_{extra_preprocessing_type}_baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    # args.extra_preprocessing_type = "rgb-grayscale"
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_rgb-grayscale_baseline/final.pt
Custom_normalization: Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.16it/s]


0.99946
0.9274
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_rgb-grayscale_mean_0.774_0.439_0.859/final.pt
Custom_normalization: Normalize(mean=[0.774, 0.439, 0.859], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.99868
0.9248
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_rgb-grayscale_mean_0.697_0.094_0.976/final.pt
Custom_normalization: Normalize(mean=[0.697, 0.094, 0.976], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.99956
0.9278
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_rgb-grayscale_std_0.1971_0.1935_0.2116/final.pt
Custom_normalization: Normalize(mean=(0.4914, 0.4822, 0.4465), std=[0.1971, 0.1935, 0.2116])
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


0.99956
0.9259
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_rgb-grayscale_std_0.2221_0.2185_0.2366/final.pt
Custom_normalization: Normalize(mean=(0.4914, 0.4822, 0.4465), std=[0.2221, 0.2185, 0.2366])
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


0.99956
0.9241


In [6]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_rgb-grayscale_baseline',
              {'test_acc': 0.9303, 'train_acc': 0.99998}),
             ('model_rgb-grayscale_mean_0.774_0.439_0.859',
              {'test_acc': 0.928, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.697_0.094_0.976',
              {'test_acc': 0.928, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_mean_0.761_0.786_0.128',
              {'test_acc': 0.9285, 'train_acc': 0.99992}),
             ('model_rgb-grayscale_std_0.3471_0.3435_0.3616',
              {'test_acc': 0.9326, 'train_acc': 0.99992}),
             ('model_rgb-grayscale_std_0.2971_0.2935_0.3116',
              {'test_acc': 0.928, 'train_acc': 1.0}),
             ('model_rgb-grayscale_std_0.2721_0.2685_0.2866',
              {'test_acc': 0.9282, 'train_acc': 0.99996}),
             ('model_rgb-grayscale_std_0.1471_0.1435_0.1616',
              {'test_acc': 0.9306, 'train_acc': 0.99998}),
             ('model_rgb-grayscale_std_0.1971_0.1935_0.2116',

In [7]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_std_{std_str}"
    else:
        label = f"model_baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_baseline/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.9998
0.941
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_mean_0.774_0.439_0.859/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.71242
0.6857
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_mean_0.697_0.094_0.976/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.9985
0.9322
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_std_0.1971_0.1935_0.2116/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.99996
0.9423
/data/weijing/DI-ting_normalization/models/CIFAR10_diff_norm_value_n_grayscale/model_std_0.2221_0.2185_0.2366/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.99998
0.9401


In [8]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_baseline', {'test_acc': 0.941, 'train_acc': 0.9998}),
             ('model_mean_0.774_0.439_0.859',
              {'test_acc': 0.6857, 'train_acc': 0.71242}),
             ('model_mean_0.697_0.094_0.976',
              {'test_acc': 0.4244, 'train_acc': 0.43628}),
             ('model_mean_0.761_0.786_0.128',
              {'test_acc': 0.5589, 'train_acc': 0.58896}),
             ('model_std_0.3471_0.3435_0.3616',
              {'test_acc': 0.8957, 'train_acc': 0.96644}),
             ('model_std_0.2971_0.2935_0.3116',
              {'test_acc': 0.9259, 'train_acc': 0.99402}),
             ('model_std_0.2721_0.2685_0.2866',
              {'test_acc': 0.9344, 'train_acc': 0.99824}),
             ('model_std_0.1471_0.1435_0.1616',
              {'test_acc': 0.9322, 'train_acc': 0.9985}),
             ('model_std_0.1971_0.1935_0.2116',
              {'test_acc': 0.9423, 'train_acc': 0.99996}),
             ('model_std_0.2221_0.2185_0.2366',
              {'test_acc': 0